In [1]:
# #turn off gpu support, no need for it
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [9]:
from custom_models import *
from inception_utils import *
from image_processing import *
from video_processing import *
import pickle
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import time

In [3]:
def get_predictions(model, model_flow, num_labels, classes_dictionary,  w, fnb):    
    start = time.time()

    root_path1 = 'full_videos_dataset/frames/'
    root_path2 = 'full_videos_dataset/flow_images/'

    print('\n\nCurrently getting predictions with weighted average ( rgb =',w[0],', flow =',w[1],' )')

    conf_mat = np.zeros((num_labels,num_labels))
    top5_correct = 0
    vids_per_class = np.zeros(num_labels) #keep it for later stats extraction

    pred_out = open('results/single_frame_results/combo'+str(fnb)+'/single_frame_predictions_'+str(w[0])+'_'+str(w[1])+'.txt','w')

    cntr,N=0,3783
    for line in open('data_a/ucf101_split1_testVideos.txt','r'):       
        videoname1 = root_path1 + line.split()[0].split('/')[1]
        videoname2 = root_path2 + line.split()[0].split('/')[1]
        label = int(line.split()[1])

        vids_per_class[label] +=1

        #get mean of every column (101 of them) to find later greatest value among 101 and get predicted class
        if(w[0]==0.0):
            rgb_pred = np.zeros((101))
            video2 = read_video(videoname2,'flow_images')
            flow_pred = np.mean(model_flow.predict(video2),axis=0)
            del video2
        elif(w[1]==0.0):
            video1 = read_video(videoname1,'frames')
            rgb_pred = np.mean(model.predict(video1),axis=0)
            flow_pred = np.zeros((101))
            del video1
        else:
            video1 = read_video(videoname1,'frames')
            rgb_pred = np.mean(model.predict(video1),axis=0)
            video2 = read_video(videoname2,'flow_images')
            flow_pred = np.mean(model_flow.predict(video2),axis=0)
            del video1,video2

        weighted_pred = w[0]*rgb_pred + w[1]*flow_pred

        conf_mat[weighted_pred.argmax(),label] += 1  

        pred_out.write('{:30} was classified as : {:25}'.format(line.split()[0].split('/')[1],classes_dictionary[weighted_pred.argmax()]))#,'with max_pred={:.4f}'.format(weighted_pred.max()))
        pred_out.write('\n\nTop 5 predictions\n-----------------\n')

        ind = np.argpartition(weighted_pred, -5)[-5:]
        for i,cnt in zip(reversed(ind[np.argsort(weighted_pred[ind])]),range(5)):
            pred_out.write(str(cnt+1)+' '+classes_dictionary[i]+'\n')#, weighted_pred[i])
            if(i==label):
                top5_correct += 1
        pred_out.write('\n'+'-'*73+'\n')

        cntr+=1
        print('{:.2f}'.format(cntr/N*100), end='',flush=True)
        print('\r', end='')
        del rgb_pred,flow_pred,weighted_pred
    pred_out.close()

    #######################
    accuracy = np.trace(conf_mat) / np.sum(conf_mat)
    print('\tAccuracy is {:.3f}'.format(accuracy*100),'%')

    top5_accuracy = top5_correct / np.sum(conf_mat)
    print('\tTop-5 Accuracy is {:.3f}'.format(top5_accuracy*100),'%')

    #######################
    df_cm = pd.DataFrame(conf_mat, index = [classes_dictionary[i] for i in ([i for i in range(num_labels)])],
                                 columns = [classes_dictionary[i] for i in ([i for i in range(num_labels)])])
    sn.set(rc={'figure.figsize':(50,50)})
    sn.heatmap(df_cm, annot=True).figure.savefig('results/single_frame_results/combo'+str(fnb)+'/single_frame_conf_mat_'+str(w[0])+'_'+str(w[1])+'.png', dpi=100)
    plt.close() 
    
    sn.set(rc={'figure.figsize':(30,30)})    
    sn.heatmap(df_cm, annot=False).figure.savefig('results/single_frame_results/combo'+str(fnb)+'/single_frame_conf_mat_no_annot_'+str(w[0])+'_'+str(w[1])+'.png', dpi=100)
    plt.close() 
    
    np.save('results/single_frame_results/combo'+str(fnb)+'/conf_mat_'+str(w[0])+'_'+str(w[1])+'.npy',conf_mat)
    
    #######################
    stats_out = open('results/single_frame_results/combo'+str(fnb)+'/single_frame_stats_'+str(w[0])+'_'+str(w[1])+'.txt','w')    
    for i in range(num_labels):
        stats_out.write('Classified correctly '+str(int(conf_mat[i,i]))+' of '+str(int(vids_per_class[i]))+' for class '+classes_dictionary[i]+'\n')
    stats_out.close()

    print('\tPartial execution time {:.2f}'.format(time.time()-start))
    return conf_mat, accuracy, top5_accuracy

In [10]:
def get_predictions_combo(input_shape, num_labels, weights, rgb_p, flow_p, fnb):
    start = time.time()

    if('inception' in rgb_p):
        model = get_model(weights='imagenet',input_shape=input_shape,num_labels=num_labels)
    else:
        model = CaffeDonahueFunctional(input_shape=input_shape,num_labels=num_labels)

    model.load_weights(rgb_p)

    model_flow = CaffeDonahueFunctional(input_shape=input_shape,num_labels=num_labels)
    model_flow.load_weights(flow_p)
    
    conf_mat={}
    acc={}
    top5acc={}
    
    conf_mat[0], acc[0], top5acc[0] = get_predictions(model, model_flow, num_labels, classes_dictionary,  weights[0], fnb)
    
    conf_mat[1], acc[1], top5acc[1] = get_predictions(model, model_flow, num_labels, classes_dictionary,  weights[1], fnb)
    
    conf_mat[2], acc[2], top5acc[2] = get_predictions(model, model_flow, num_labels, classes_dictionary,  weights[2], fnb)
    
    conf_mat[3], acc[3], top5acc[3] = get_predictions(model, model_flow, num_labels, classes_dictionary,  weights[3], fnb)
    
    conf_mat[4], acc[4], top5acc[4] = get_predictions(model, model_flow, num_labels, classes_dictionary,  weights[4], fnb)
    
    print('\n\nExecution time {:.2f}'.format(time.time()-start))
    return conf_mat, acc, top5acc

In [5]:
input_shape = (crop_height, crop_width, 3)

classes_dictionary = pickle.load(open('data_a/action_dictionary.p','rb'))

rgb_path = ['saved_models/caffenet_single_rgb.hdf5','saved_models/inception_rgb.hdf5']
flow_path = ['saved_models/caffenet_single_flow.hdf5']


weights0 = [1.0, 0.0]
weights1 = [0.67, 0.33]
weights2 = [0.5, 0.5]
weights3 = [0.33, 0.67]
weights4 = [0.0, 1.0]

weights={}
weights[0]=weights0
weights[1]=weights1
weights[2]=weights2
weights[3]=weights3
weights[4]=weights4

In [6]:
# CaffenetRGB + CaffenetFLOW
conf_mat0, acc0, top5acc0 = get_predictions_combo(input_shape, num_labels, weights, rgb_path[0], flow_path[0], 0)



Currently getting predictions with weighted average ( rgb = 1.0 , flow = 0.0  )
	 Accuracy is 48.929 %
	 Top-5 Accuracy is 74.253 %
	 Partial execution time 3802.45


Currently getting predictions with weighted average ( rgb = 0.67 , flow = 0.33  )
	 Accuracy is 52.683 %
	 Top-5 Accuracy is 81.338 %
	 Partial execution time 7448.36


Currently getting predictions with weighted average ( rgb = 0.5 , flow = 0.5  )
	 Accuracy is 57.098 %
	 Top-5 Accuracy is 81.549 %
	 Partial execution time 7239.12


Currently getting predictions with weighted average ( rgb = 0.33 , flow = 0.67  )
	 Accuracy is 59.107 %
	 Top-5 Accuracy is 81.549 %
	 Partial execution time 7461.92


Currently getting predictions with weighted average ( rgb = 0.0 , flow = 1.0  )
	 Accuracy is 48.454 %
	 Top-5 Accuracy is 74.227 %
	 Partial execution time 3711.94


Execution time 29665.42


In [11]:
# InceptionRGB + CaffenetFLOW
conf_mat1, acc1, top5acc1 = get_predictions_combo(input_shape, num_labels, weights, rgb_path[1], flow_path[0], 1)



Currently getting predictions with weighted average ( rgb = 1.0 , flow = 0.0  )
	 Accuracy is 67.195 %
	 Top-5 Accuracy is 86.624 %
	 Partial execution time 6388.06


Currently getting predictions with weighted average ( rgb = 0.67 , flow = 0.33  )
	 Accuracy is 69.997 %
	 Top-5 Accuracy is 88.210 %
	 Partial execution time 9994.35


Currently getting predictions with weighted average ( rgb = 0.5 , flow = 0.5  )
	 Accuracy is 70.632 %
	 Top-5 Accuracy is 88.158 %
	 Partial execution time 9896.20


Currently getting predictions with weighted average ( rgb = 0.33 , flow = 0.67  )
	 Accuracy is 68.623 %
	 Top-5 Accuracy is 87.787 %
	 Partial execution time 9985.23


Currently getting predictions with weighted average ( rgb = 0.0 , flow = 1.0  )
	 Accuracy is 48.454 %
	 Top-5 Accuracy is 74.227 %
	 Partial execution time 3710.90


Execution time 39989.04
